In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307944  [    0/60000]
loss: 2.289771  [ 6400/60000]
loss: 2.276964  [12800/60000]
loss: 2.273931  [19200/60000]
loss: 2.254935  [25600/60000]
loss: 2.225292  [32000/60000]
loss: 2.226002  [38400/60000]
loss: 2.195939  [44800/60000]
loss: 2.197852  [51200/60000]
loss: 2.161490  [57600/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.162308 

Epoch 2
-------------------------------
loss: 2.172738  [    0/60000]
loss: 2.159805  [ 6400/60000]
loss: 2.109092  [12800/60000]
loss: 2.133177  [19200/60000]
loss: 2.087056  [25600/60000]
loss: 2.019466  [32000/60000]
loss: 2.046275  [38400/60000]
loss: 1.972619  [44800/60000]
loss: 1.980005  [51200/60000]
loss: 1.903456  [57600/60000]
Test Error: 
 Accuracy: 59.8%, Avg loss: 1.912348 

Epoch 3
-------------------------------
loss: 1.940260  [    0/60000]
loss: 1.909830  [ 6400/60000]
loss: 1.801057  [12800/60000]
loss: 1.849282  [19200/60000]
loss: 1.744036  [25600/60000]
loss: 1.681083  [32000/600